In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# （コア数128＆異なる問題サイズでの結果）OR（異なるコア数＆異なる問題サイズでの結果）

In [2]:


resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
date = "2021年11月17日"

resultDict128WithoutInit = {}
resultDict128WithInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="128WithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="128WithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).copy
)
#

# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )

    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

% _____benchmarkName=cg_____
% _____benchmarkName=ep_____
% _____benchmarkName=ft_____
% _____benchmarkName=is_____
% _____benchmarkName=lu_____
% _____benchmarkName=mg_____
\begin{table}
\centering
\caption{初期変数を含めて構築したモデルの平均MAPE}
\label{128WithInit}
\begin{tabular}{lr}
\toprule
{} &  平均絶対誤差 \\
\midrule
CG &     0.0 \\
EP &     0.0 \\
FT &     0.0 \\
IS &     0.0 \\
LU &     1.6 \\
MG &     0.0 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{初期変数を含めずに構築したモデルの平均MAPE}
\label{128WithoutInit}
\begin{tabular}{lr}
\toprule
{} &  平均絶対誤差 \\
\midrule
CG &    71.0 \\
EP &     0.0 \\
FT &    50.2 \\
IS &     0.0 \\
LU &    30.1 \\
MG &    10.4 \\
\bottomrule
\end{tabular}
\end{table}

% _____benchmarkName=cg_____
% _____benchmarkName=ep_____
% _____benchmarkName=ft_____
% _____benchmarkName=is_____
% _____benchmarkName=lu_____
% _____benchmarkName=mg_____
\begin{table}
\centering
\caption{初期変数を含めて構築したモデルの平均MAPE}
\label{AllWithInit}
\begin{tabular}{lr}
\toprule
{} &  平

# （コア数128＆異なる問題サイズでの結果）OR（異なるコア数＆異なる問題サイズでの結果）

## 「コア数に線形モデル、問題サイズに反比例モデル」となる新モデルを追加

In [3]:


resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelLinAndIp": "線形および反比例モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
date = "2021年12月25日"

resultDict128WithoutInit = {}
resultDict128WithInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="128WithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="128WithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).copy
)
#

# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )

    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

% _____benchmarkName=cg_____


KeyError: '.TAU_application'